# Análise de Dados Exploratória do Titanic

![titanic](img/titanic_historical.jpg "Pintura do Titanic")

# 1. Introdução

O RMS Titanic foi um navio de passageiros britânico operado pela empresa *White Star Line* que afundou no Oceano Atlântico Norte nas primeiras horas da manhã de 15 de abril de 1912, depois de atingir um iceberg durante sua viagem inaugural de Southampton a Nova York. Dos 2.224 passageiros e tripulantes a bordo, mais de 1.500 morreram, tornando o naufrágio um dos desastres marítimos comerciais mais mortais da história moderna em tempos de paz.

Neste estudo iremos explorar um conjunto de dados baseado na lista de passageiros do Titanic editada por Michael A. Findlay, publicada originalmente em Eaton & Haas (1994) Titanic: Triumph and Tragedy, Patrick Stephens Ltd, e expandida com a ajuda da comunidade da Internet. Os arquivos HTML originais foram obtidos por Philip Hind (1999) e o *dataset* descreve o status de sobrevivência de passageiros individuais no Titanic. Ele não contém informações para a tripulação, mas contém idades atuais e estimadas para quase 80% dos passageiros.

## 1.1. Objetivos

O objetivo deste estudo é realizar uma Análise Exploratória dos Dados (Exploratory Data Analysis - EDA) do conjunto de dados `titanic3`, adaptado pelo autor e disponível em [data/titanic3.csv](data/titanic.csv); a fim de caracterizar o perfil dos sobreviventes do fatídico naufrágio. Especificiamente serão respondidas as seguintes questões de pesquisa:

1. [Qual é o número de Sobreviventes?](#3.1.-Qual-é-o-número-de-Sobreviventes?)
2. [Homens ou mulheres tiveram mais chances de sobreviver?](#3.2.-Homens-ou-mulheres-tiveram-mais-chances-de-sobreviver?)
3. [Qual classe social teve mais sobreviventes?](#3.3.-Qual-classe-social-teve-mais-sobreviventes?)
4. [Qual faixa etária teve mais chances de sobreviver?](#3.4.-Qual-faixa-etária-teve-mais-chances-de-sobreviver?)


# 2. Metodologia

Nesta seção será apresentado todo o processo de preparação, organização e limpeza de dados feito no *dataset* que possui os seguintes dados:

| Coluna          | Descrição                                                                 |
|-----------------|---------------------------------------------------------------------------|
| id_passageiro   | Identficador único do passageiro.                                         |
| classe          | Classe social.                                                            |
| sobreviveu      | Sobrevivente? Sim (1), Não (0).                                           |
| nome            | Nome do passageiro.                                                       |
| sexo            | Masculino (male), Feminino (female).                                      |
| idade           | Idade do passageiro.                                                      |
| irmaos_conjuges | Número de irmãos e cônjuges a bordo.                                      |
| pais_filhos     | Número de pais e filhos a bordo.                                          |
| bilhete         | Número do bilhete                                                         |
| tarifa          | Preço da tarifa do passageiro.                                            |
| cabine          | Cabine.                                                                   |
| embarque        | Nome do porto de embarque: C = Cherbourg; Q = Queenstown; S = Southampton |
| bote            | Bote salva vidas.                                                         |
| corpo           | Número de identificação do corpo.                                         |
| destino         | Local de desembarque do passageiro.                                       |

## OBSERVAÇÕES

- `classe` é uma aproximação do status socioeconômico na época, onde: 1 = Classe Alta1; 2 = Classe Média e 3 = Classe Baixa;
- `idade` está representada em anos, porém, se a idade for menor que Um (1) ou caso tenha sido estimada, ela estará com casas decimais xx.5;
- `tarifa` está em Libras esterlinas (British Pounds - £) anteriores a 1970;
- `irmaos_conjuges` e `pais_filhos`: as variáveis de relação familiar de algumas relações foram ignoradas; a seguir estão as definições usadas:
    - **Irmão**: Irmão, irmã, meio-irmão ou meia-irmã do passageiro a bordo do Titanic;
    - **Cônjuge**: Marido ou esposa do passageiro a bordo do Titanic (amantes e noivos ignorados);
    - **Pai**: Mãe ou pai do passageiro a bordo do Titanic;
    - **Criança**: Filho, Filha, Enteado ou Enteada do Passageiro a bordo do Titanic;
    - Outros parentes excluídos deste estudo incluem primos, sobrinhos / sobrinhas, tias / tios e parentes;
    - Algumas crianças viajavam apenas com uma babá, portanto foi atribuído 0 para elas em pais_filhos; 
    - Alguns viajaram com amigos ou vizinhos muito próximos em uma vila, no entanto, as definições não apóiam essas relações.


## 2.1. Configuração do Ambiente

In [2]:
import pandas as pd
import numpy as np

## 2.2. Leitura dos Dados

In [8]:
titanicDf = pd.read_csv('/titanic3.csv');

## 2.3. Organização e Limpeza dos Dados

### 2.3.1. Visão Geral do Conjunto de Dados

In [9]:
titanicDf.head()

,id_passageiro,classe,sobreviveu,nome,sexo,idade,irmaos_conjuges,pais_filhos,bilhete,tarifa,cabine,embarque,bote,corpo,destino
0,1,1,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,2,1,1,"Allison, Master. Hudson Trevor",male,1.0,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1,0,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


### 2.3.2 Remoção de Colunas
Remoção de colunas incompletas e/ou desnecessárias para a análise.

In [11]:
titanicDf = titanicDf.drop(columns=["id_passageiro", "corpo", "nome"], axis=1)
titanicDf.head()

,classe,sobreviveu,sexo,idade,irmaos_conjuges,pais_filhos,bilhete,tarifa,cabine,embarque,bote,destino
0,1,1,female,29.0,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,1.0,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"


### 2.3.3. Dados Ausentes

#### 2.3.3.1 Idades Ausentes
Para os dados ausentes da coluna `idade` será feita a substituição dos dados vazios pela média das idades na época.

In [12]:
def getIdade(linha):
  if linha["idade"] == "":
    return linha["idade"].mean()

titanicDf.apply(getIdade, axis=1)
titanicDf.head()

,classe,sobreviveu,sexo,idade,irmaos_conjuges,pais_filhos,bilhete,tarifa,cabine,embarque,bote,destino
0,1,1,female,29.0,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,1.0,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"


#### 2.3.3.2 Tarifas Ausentes
Para os dados ausentes da coluna `tarifa`será feita a substituição dos dados vazios pela média de preço das tarifas na época.

In [14]:
def tarifaAusente(linha):
  if linha["tarifa"] == "":
    return linha["tarifa"].mean()

titanicDf.apply(tarifaAusente, axis=1)
titanicDf.head()

,classe,sobreviveu,sexo,idade,irmaos_conjuges,pais_filhos,bilhete,tarifa,cabine,embarque,bote,destino
0,1,1,female,29.0,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,1.0,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"


#### 2.3.3.3 Portos de Embarque Ausentes
Para os dados ausentes da coluna `embarque`será feita a remoção dos dados vazios.

In [15]:
def embarquesAusentes(embarque):
  return embarque != ""

titanicDf[titanicDf["embarque"].apply(embarquesAusentes)]

,classe,sobreviveu,sexo,idade,irmaos_conjuges,pais_filhos,bilhete,tarifa,cabine,embarque,bote,destino
0,1,1,female,29.0,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,1.0,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,female,15.0,1,0,2665,14.4542,NaN,C,NaN,NaN
1305,3,0,female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN
1306,3,0,male,27.0,0,0,2656,7.2250,NaN,C,NaN,NaN
1307,3,0,male,27.0,0,0,2670,7.2250,NaN,C,NaN,NaN


## 2.4. Mapeamento de Dados
Para melhorar a análise e interpretação dos dados os seguintes atributos serão modificados:

| Coluna    | Original     | Modificado                          |
|:----------|:-------------|:------------------------------------|
| Classe    | 1, 2, 3      | Alta, Média, Baixa                  |
| Sexo      | male, female | M, F                                |
| Embarque  | C, Q, S      | Cherbourg, Queenstown, Southampton  |


## 2.5.  Feature Engineering

A engenharia de recursos (*Feature Engineering*) é o processo de usar o conhecimento do domínio para extrair recursos dos dados brutos. Neste estudo serão acrescentadas as seguintes colunas para enriquecer a análise de dados:

| Coluna         | Descrição                                                         |
|:---------------|:------------------------------------------------------------------|
| total_parentes | Número total de parentes a bordo: `irmaos_conjuges + pais_filhos` |
| faixa_etaria   | Intervalo de idades dos passageiros.                              |

### 2.5.1 Total de Parentes

In [16]:
def somar(linha):
  return linha["irmaos_conjuges"] + linha["pais_filhos"]

titanicDf["total"] = titanicDf.apply(somar, axis=1)
titanicDf.head()

,classe,sobreviveu,sexo,idade,irmaos_conjuges,pais_filhos,bilhete,tarifa,cabine,embarque,bote,destino,total
0,1,1,female,29.0,0,0,24160,211.3375,B5,S,2,"St Louis, MO",0
1,1,1,male,1.0,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON",3
2,1,0,female,2.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON",3
3,1,0,male,30.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON",3
4,1,0,female,25.0,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON",3


### 2.5.2 Faixas Etárias

# 3. Análise dos Dados

## 3.1. Qual é o número de Sobreviventes?

In [17]:
sobreviventes = titanicDf.query("sobreviveu == 1")
sobreviventes.shape[0]

500

## 3.2. Homens ou mulheres tiveram mais chances de sobreviver?

## 3.3. Qual classe social teve mais sobreviventes?

## 3.4. Qual faixa etária teve mais chances de sobreviver?

# REFERÊNCIAS

- [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data)
- [Encyclopedia Titanica](https://www.encyclopedia-titanica.org/)
- [Basic Feature Engineering with the Titanic Data](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)
- Hind, Philip.  "Encyclopedia Titanica."  Online.  Internet. n.p.  02 Aug 1999.